In [39]:
from azureml.core import Workspace, Environment, Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace, Webservice

ws = Workspace.from_config()
service_name = "catboost-endpoint"

try:
    service = Webservice(ws, service_name)
    service.delete()
    print(f"🗑️ Deleted existing service: {service_name}")
except:
    print(f"⚠️ No existing service found: {service_name}")

# ✅ Confirm model exists
model_name = "catboost-thrombosis-predictor"
model = Model(ws, model_name)
print(f"✅ Found registered model: {model_name}")


⚠️ No existing service found: catboost-endpoint
✅ Found registered model: catboost-thrombosis-predictor


In [35]:
env = Environment(name="catboost-env")
env.python.conda_dependencies.add_pip_package("catboost")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn")
env.python.conda_dependencies.add_pip_package("pandas")
env.python.conda_dependencies.add_pip_package("azureml-defaults")
print("✅ Environment created successfully")


✅ Environment created successfully


In [36]:
inference_config = InferenceConfig(
    entry_script="score.py",  # Ensure this file exists
    environment=env
)
print("✅ Inference configuration created")


✅ Inference configuration created


In [37]:
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    auth_enabled=False
)
print("✅ Deployment configuration set")


✅ Deployment configuration set


In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
print("✅ Azure ML Workspace Loaded:", ws.name)


✅ Azure ML Workspace Loaded: mlops-simple-flow


In [2]:
from azureml.core.model import Model

model = Model.register(
    workspace=ws,
    model_name="catboost-thrombosis-predictor",
    model_path="catboost_model.cbm",  # Path to your model file
    description="CatBoost model for predicting thrombotic events",
)

print("✅ Model registered successfully!")


Registering model catboost-thrombosis-predictor
✅ Model registered successfully!


In [3]:
env_yaml = """name: catboost-env
dependencies:
  - python=3.8
  - pip:
    - azureml-defaults
    - catboost
    - numpy
    - pandas
"""
with open("environment.yml", "w") as f:
    f.write(env_yaml)

print("✅ environment.yml created successfully!")


✅ environment.yml created successfully!


In [4]:
from azureml.core import Environment

env = Environment.from_conda_specification(name="catboost-env", file_path="environment.yml")
env.register(workspace=ws)

print("✅ Environment registered successfully!")


✅ Environment registered successfully!


In [5]:
score_script = """import json
import numpy as np
import pandas as pd
import joblib
from azureml.core.model import Model
from catboost import CatBoostClassifier

def init():
    global model
    model_path = Model.get_model_path("catboost-thrombosis-predictor")
    model = CatBoostClassifier()
    model.load_model(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)["data"]
        df = pd.DataFrame(data)
        
        # Convert categorical features to int if necessary
        categorical_features = ["gender", "work_type", "Residence_type", "smoking_status"]
        for col in categorical_features:
            if col in df.columns:
                df[col] = df[col].astype(str).astype("category")

        predictions = model.predict(df)
        return {"predictions": predictions.tolist()}
    except Exception as e:
        return {"error": str(e)}
"""

with open("score.py", "w") as f:
    f.write(score_script)

print("✅ score.py created successfully!")


✅ score.py created successfully!


In [6]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

# Define inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",  # The script handling inference requests
    environment=env
)

# Define ACI deployment configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy the model
service_name = "catboost-endpoint"
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config
)

service.wait_for_deployment(show_output=True)
print(f"✅ Model deployed successfully! Endpoint URL: {service.scoring_uri}")


/tmp/ipykernel_5542/1950278768.py:15: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(
Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: a05c6fd7-73db-48b0-8ef0-a2a0d12c0b7c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'joblib', please r

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-02-01 15:16:02+00:00 Creating Container Registry if not exists.
2025-02-01 15:16:03+00:00 Use the existing image.
2025-02-01 15:16:03+00:00 Generating deployment configuration.
2025-02-01 15:16:05+00:00 Submitting deployment to compute.
2025-02-01 15:16:14+00:00 Checking the status of deployment catboost-endpoint..
2025-02-01 15:17:51+00:00 Checking the status of inference endpoint catboost-endpoint.
Failed


WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Unhealthy
Operation ID: a05c6fd7-73db-48b0-8ef0-a2a0d12c0b7c
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ModuleNotFoundError: No module named 'joblib', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ModuleNotFoundError: No module named 'joblib', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Unhealthy\nOperation ID: a05c6fd7-73db-48b0-8ef0-a2a0d12c0b7c\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\":

In [ ]:
# Replace with actual endpoint
endpoint = service.scoring_uri
print(endpoint)

In [ ]:
import requests
import json

# Replace with actual endpoint
endpoint = service.scoring_uri

test_data = {
    "data": [
        {
            "gender": 1,
            "age": 60,
            "hypertension": 1,
            "heart_disease": 0,
            "ever_married": 1,
            "work_type": 2,
            "Residence_type": 1,
            "avg_glucose_level": 85.6,
            "bmi": 28.9,
            "smoking_status": 0
        }
    ]
}

headers = {"Content-Type": "application/json"}
response = requests.post(endpoint, data=json.dumps(test_data), headers=headers)
print("Response from Model:", response.json())
